<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/4/47/Acronimo_y_nombre_uc3m.png" width=50%/>

<h1><font color='#12007a'>Procesamiento de Lenguaje Natural con Aprendizaje Profundo</font></h1>
<p>Autora: Isabel Segura Bedmar</p>

<img align='right' src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" width=15%/>
</center>  

# Práctica 1 - EDOS (parte 4: transformers entrenados con textos sintéticos augmentation)

(EDOS)


En este notebook, exploramos distintos transformers para las tareas de EDOS (SemEval 2023 - Task 10 - Explainable Detection of Online Sexism). En concreto lo que haremos en este notebook es entrenar un transformer con datos sintéticos, además de con el conjunto de entrenamiento.

Recuerda que el conjunto de datos para la tarea EDOS (https://github.com/rewire-online/edos).



**NOTA PARA PODER EJECUTAR ESTE NOTEBOOK**:

1) Para poder ejercutar correctamente este notebook, deberás abrirlo en tu Google Drive (por ejemplo, en la carpeta 'Colab Notebooks').

2) Además, debes guardar el dataset en tu Google Drive, dentro de carpeta 'Colab Notebooks/data/edos/'.



En concreto, vamos a comparar los siguientes transformers, y los aplicaremos a cada una de las tres tareas:


In [1]:
models = ['bert-base-uncased', 'bert-base-cased',
          'distilbert-base-uncased', 'distilbert-base-cased',
          'roberta-base',
          'xlnet-base-cased',
          'gpt2', 't5-small']

model_name = models[0]

TASK = "b" # a, b or c
TASK = TASK.lower()

print(model_name, TASK)


bert-base-uncased b


## Instalar librerías

Como siempre tenemos que instalar las librerías de transformers y datasets:

In [2]:
import torch
torch.__version__

'2.0.1+cu118'

In [3]:
!pip install -q transformers[torch] datasets


Para garantizar la reproductividad de nuestros experimentos vamos a fijar una semilla. Con esto conseguimos que las inicializaciones siempre sean las mismas:

In [4]:
from transformers import set_seed
set_seed(42)

## Dataset

Vamos a cargar el dataset, tal y como ya hicimos en la primera parte de esta práctica:


In [5]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/Colab Notebooks/data/edos/')

import pandas as pd
df = pd.read_csv("edos_labelled.csv")
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,rewire_id,text,label_sexist,label_category,label_vector,split
0,sexism2022_english-9609,"In Nigeria, if you rape a woman, the men rape ...",not sexist,none,none,dev
1,sexism2022_english-16993,"Then, she's a keeper. 😉",not sexist,none,none,train
2,sexism2022_english-13149,This is like the Metallica video where the poo...,not sexist,none,none,train
3,sexism2022_english-13021,woman?,not sexist,none,none,train
4,sexism2022_english-966,I bet she wished she had a gun,not sexist,none,none,dev


Cargamos cada split en un dataframe distinto. Estos dataframe serán transformados en objetos Dataset y usados para construir un DatasetDict que albergue los tres splits de forma conjunta:

In [6]:
df_train_original = df[df['split']=='train']
df_dev = df[df['split']=='dev']
df_test = df[df['split']=='test']

También vamos a cargar un conjunto de instancias cuyos textos fueron generados de forma automática a partir de la librería textaugment:

In [7]:
df_syn = pd.read_csv("edos_aug_eda.csv")
df_syn = df_syn.drop(columns=['__index_level_0__'])
df_syn.head()

,rewire_id,text,label_sexist,label_category,label_vector,split
0,sexism2022_english-16993,"Then, she's a keeper. 😉",not sexist,none,none,train
1,sexism2022_english-13149,This is like the Metallica video where the poo...,not sexist,none,none,train
2,sexism2022_english-13021,woman?,not sexist,none,none,train
3,sexism2022_english-14998,Unlicensed day care worker reportedly tells co...,not sexist,none,none,train
4,sexism2022_english-7228,[USER] Leg day is easy. Hot young lady who wea...,sexist,3. animosity,3.3 backhanded gendered compliments,train


Ahora vamos a concatenar el conjunto de training original y el conjunto que acabamos de cargar con los textos sintéticos:

In [8]:
df_train = pd.concat([df_train_original, df_syn])
print(df_train_original.shape)
print(df_train.shape)


(14000, 6)
(28000, 6)


Ahora ya podemos transformar los tres dataframes a objetos Dataset y crear el objeto DatasetDict, que nos facilitará procesar y pasar los datos al modelo transformer:

In [9]:
# de dataframes a dataset
from datasets import DatasetDict, Dataset
dict_dataset= DatasetDict()
dict_dataset['train'] = Dataset.from_pandas(df_train )
dict_dataset['validation'] = Dataset.from_pandas(df_dev)
dict_dataset['test'] = Dataset.from_pandas(df_test)

# borramos las columnas que nos vamos a utilizar
dict_dataset=dict_dataset.remove_columns(['split','__index_level_0__'])

dict_dataset


DatasetDict({
    train: Dataset({
        features: ['rewire_id', 'text', 'label_sexist', 'label_category', 'label_vector'],
        num_rows: 28000
    })
    validation: Dataset({
        features: ['rewire_id', 'text', 'label_sexist', 'label_category', 'label_vector'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['rewire_id', 'text', 'label_sexist', 'label_category', 'label_vector'],
        num_rows: 4000
    })
})

Para las tareas B y C (clasificación y clasificación fina de mensajes sexistas) deberemos eliminar todos los mensajes que no están clasificados con la clase 'sexist' en el campo 'label_sexist':

In [10]:
if TASK != 'a':
    dict_dataset = dict_dataset.filter(lambda example: example["label_sexist"]=='sexist')

# Cada tarea, tiene un nombre de label distinto:
label_task = {'a': 'label_sexist', 'b': 'label_category', 'c':'label_vector'}

# Borramos todas las columnas excepto el texto y el campo label correspondiente a la tarea
columns_to_remove = list(set(dict_dataset['train'].features) - set(['text', label_task[TASK]]))
dict_dataset = dict_dataset.remove_columns(columns_to_remove)
# renombramos el campo label específico de la tarea a label
print('renombramos: ', label_task[TASK])
dict_dataset = dict_dataset.rename_column(label_task[TASK],'label')
dict_dataset

Filter:   0%|          | 0/28000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4000 [00:00<?, ? examples/s]

renombramos:  label_category


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 6796
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 486
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 970
    })
})

### Label encoding

Las labels en este dataset son textos. Así, por ejemplo, para la tarea A, las labels son: 'not sexist' y 'sexist'.
Para la tarea B, las labels son:
- 1. threats, plans to harm and incitement
- 2. derogation
- 3. animosity
- 4. prejudiced discussions

Y para la tarea C, las labels son:
- '1.1 threats of harm'',
-  '1.2 incitement and encouragement of harm',
-  '2.1 descriptive attacks',
-  '2.2 aggressive and emotive attacks',
-  '2.3 dehumanising attacks & overt sexual objectification',
-  '3.1 casual use of gendered slurs, profanities, and insults',
-  '3.2 immutable gender differences and gender stereotypes',
-  '3.3 backhanded gendered compliments',
-  '3.4 condescending explanations or unwelcome advice',
-  '4.1 supporting mistreatment of individual women',
-  '4.2 supporting systemic discrimination against women as a group'

Las labels deben ser transformadas en números enteros. Para ello usaremos la clase LabelEncoder de sklearn:

In [11]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# entrenamos el codificador de etiquetas
le.fit(dict_dataset['train']['label'])
# guardamos el conjunto de labels y su número
LABELS = list(le.classes_)
NUM_LABELS = len(LABELS)

# transformamos las etiquetas de texto a número en los tres splits
# las guardamos en tres listas
y_train = le.transform(dict_dataset['train']['label'])
y_val = le.transform(dict_dataset['validation']['label'])
y_test = le.transform(dict_dataset['test']['label'])




Vamos a comprobar que en efecto la transformación se ha hecho de forma correcta. Tomamos, por ejemplo, las 5 primero labels del conjunto test y vemos que se han codificado correctamente:

In [12]:
print(dict_dataset['test']['label'][:5])
print(y_test[:5])

['2. derogation', '2. derogation', '2. derogation', '2. derogation', '2. derogation']
[1 1 1 1 1]


## Tokenización

Siempre es necesario cargar el tokenizador asociado con el transformer que vayamos a utilizar.


In [13]:
if 'uncased' in model_name:
        do_lower_case = True
else:
        do_lower_case = False

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=do_lower_case)


In [14]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=NUM_LABELS)
print(model_name, ' cargado')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


bert-base-uncased  cargado


### Tamaño de los textos

Aunque ya hemos estudiamos la distribución de los textos en el notebook anterior (y vimos que el tamaño medio de los textos es de unos 22 tokens), vamos a mirar qué tamaño es el del texto más largo. En este caso, ya sí vamos a usar el tokenizador:

In [15]:
# obtenemos la longitud de la secuencia más larga en el conjunto de training
MAX_LENGTH = max([len(tokenizer(text).tokens())  for text in dict_dataset['train']['text']])
print(MAX_LENGTH)


93


Vemos que el tamaño máximo es de 93 tokens. En realidad, en esos 93 tokens habrá tokens y también subtokens, porque los tokenizadores de los transformers dividien las palabras desconocidas (es decir, aquellas que no están en la colección de textos utilizada para pre-entrenar el modelo) en subtokens para que puedan ser represenatdos.

El tamaño máximo que permiten la mayoría de los modelos es 512, así que podríamos preparar nuestras entradas para que tuvieran esa longitud, pero en realidad usando una longitud de 93, vamos a representar todas los textos de nuestro dataset, y además, vamos a ahorrar recursos.

In [16]:
MAX_LENGTH = min(MAX_LENGTH, 512)
print('MAX_LENGTH:', MAX_LENGTH)

MAX_LENGTH: 93


Una vez que ya hemos estimado el tamaño máximo de nuestras secuencias (textos) de entrada, ya podemos definir nuestra función para tokenizar todos los textos de los tres splits.

Es necesario el parámetro **truncation** igual a True, porque en el conjunto de validación o en el conjunto test, podría existir algún texto con un número mayor de tokens.



In [17]:
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding=True, max_length=MAX_LENGTH)


Una vez definida la función, la aplicamos sobre todo el dataset, gracias a la función map. Además, vamos a borrar la columna text.

La columna 'label' también la eliminamos porque sus valores están en formato string. Lo que hacemos es eliminar dicha columna y volver a añadirla en cada split, pero ahora guardando los valores de las labels ya codificados (transformados a números):

In [18]:
encoded_dataset = dict_dataset.map(tokenize, batched=True)
# quitamos text y label. label se quita porque sus valores son string
encoded_dataset = encoded_dataset.remove_columns(['text','label'])
# añadimos a cada split la columna label otra vez, pero ahora con sus correspodientes valores transformados a números (lo hicimos en Label encoding)
encoded_dataset['train'] = encoded_dataset['train'].add_column('label', y_train)
encoded_dataset['validation'] = encoded_dataset['validation'].add_column('label', y_val)
# además, borramos la codificación de test, porque realmente no se va a usar durante el entrenamiento
del(encoded_dataset['test'] )
encoded_dataset

Map:   0%|          | 0/6796 [00:00<?, ? examples/s]

Map:   0%|          | 0/486 [00:00<?, ? examples/s]

Map:   0%|          | 0/970 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'label'],
        num_rows: 6796
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'label'],
        num_rows: 486
    })
})

## Modelo

Como vimos durante los ejercicios del curso, antes de entrenar un modelo, siempre hay que proporcionar un conjunto de hiperparámetros (que siempre es recomendable probar durante la experimentación con distintos valroes para encontrar los mejores valores) y un conjunto de métricas asociado con la tarea.
En nuestro caso, como la tarea es clasificación, utilizaremos las métricas de precisión, recall y f1, y para su cálculo simplemente vamos a utilizar la librería sklearn:

In [19]:
# define metrics
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):

    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='macro')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,    #we could return just the accuracy
        'f1': f1,
        'precision': precision,
        'recall': recall
    }



Respecto a los argumentos, como ya hemos visto en otros ejercicios, usaremos la clase TrainingArguments que ya tiene los hiper-parámetros definidos e inicializados para este tipo de tareas. Trata a modificar algunos valores como el número de epochs, el ratio de aprendizaje (learning rate) o el tamaño del batch, y estudia si los resultados mejoran en el conjunto de validación.

In [20]:

from transformers import TrainingArguments
args = TrainingArguments(
    output_dir='./outputs/',
    # overwrite_output_dir = True,  # If `True`, overwrite the content of the output directory. Use this to continue training if `output_dir` points to a checkpoint directory.
    logging_dir='./logs',
    num_train_epochs=1, # usa 1 para un entrenamiento rápido, aunque lo recomendable es un valor de 3 a 5
    evaluation_strategy = "epoch",  # "steps",   evaluate each `logging_steps`, logging_steps=400,               # log & save weights each logging_steps     save_steps=400,
                                    # save_steps=400,
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",

)

Ya estamos listos para crear el objeto Trainer, al que pasamos el modelo, el tokenizador, los argumentos, la función que cálcula las métricas, y los datos codificados de training y validación:

In [21]:
from transformers import Trainer

trainer = Trainer(
    model,                                  # the model
    args,                                   # the arguments of the model
    train_dataset=encoded_dataset['train'],               # the training dataset
    eval_dataset=encoded_dataset['validation'],               #the validation dataset
    tokenizer=tokenizer,                    # the tokenizer
    compute_metrics=compute_metrics,        # the metrics used to evaluate the validation, these are calculated in each epoch
)

# training
print('Training: ', model_name)
trainer.train()

Training:  bert-base-uncased


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.883763,0.613169,0.578304,0.659439,0.540391


TrainOutput(global_step=425, training_loss=1.0010510971966913, metrics={'train_runtime': 136.4396, 'train_samples_per_second': 49.81, 'train_steps_per_second': 3.115, 'total_flos': 324797930200224.0, 'train_loss': 1.0010510971966913, 'epoch': 1.0})

Si estás entrenando el modelo BERT para la tarea a, verás que con una sola epoch, la f1 es 0.81. No está mal para una tarea de clasificación binaria.

También vemos que el uso de los textos aumentados tampoco parece mejorar los resultados notablemente (para la tarea a).


Para el resto de tareas y modelos, los resultados serán distintos.

Por ejemplo, para la tarea b, el modelo BERT con textos aumentados, obtiene una F1 de 57.8%, que es una mejora muy significativa respecto al modelo bERT que no usa textos aumentados (y que para esta tarea conseeguía una F1 de 34.2%).



En la siguiente celda, puedes evaluar el mejor modelo obtenido durante el training sobre el conjunto de validación:

In [22]:
result = trainer.evaluate()
result

{'eval_loss': 0.883762538433075,
 'eval_accuracy': 0.6131687242798354,
 'eval_f1': 0.5783037749695226,
 'eval_precision': 0.6594393923999188,
 'eval_recall': 0.5403910715813067,
 'eval_runtime': 2.3991,
 'eval_samples_per_second': 202.577,
 'eval_steps_per_second': 12.922,
 'epoch': 1.0}

### Guardar el modelo

A veces es interesante guardar el modelo para  utilizarlo más tarde para inferir la clasificación sobre nuevos textos, sin necesidad de volver a repetir el entrenamiento. En la siguiente celda, puedes ver que el modelo se almacena en la subcarpeta models (en la carpeta actual de trabajo, o podrías indicar otra ruta).
Es necesario guardar el tokenizador y el modelo.


In [23]:
import os
SAVE_MODEL = False
if SAVE_MODEL:
    models_dir = 'models/'
    if not os.path.exists(models_dir): ### If the file directory doesn't already exists,
        os.makedirs(models_dir) ### Make it please

    model_path = models_dir+model_name
    model_path += "_{}".format(TASK)

    # grabamos el modelo y el tokenizador
    tokenizer.save_pretrained(model_path)
    trainer.save_model(model_path)

Una vez entrenado el modelo, podemos usarlo sobre los textos del conjunto test para inferir sus etiquetas. Estas se comparán con las del conjunto test, y se podrán obtener los resultados finales:


In [24]:
def get_prediction(text):
    # recibe el texto y lo tokeniza usando la misma función que se uso para el training
    inputs = tokenizer(text, truncation=True, padding='max_length', max_length=MAX_LENGTH, return_tensors="pt").to("cuda")
    # aplica el modelo sobre la entrada
    outputs = model(**inputs)
    # recupera las probabilidades
    probs = outputs[0].softmax(1)
    # la función argmax nos devuelve la clase con mayor probabilidad
    # devuelve el nombre de la clase, no su codificación
    return probs.argmax().item()

Aplicamos la función get_prediction sobre todos los textos del conjunto test, y además recuperamos las labels del conjunto test:

In [25]:
y_pred=[LABELS[get_prediction(text)] for text in dict_dataset['test']['text']]


Recuperamos las labels del conjunto (sin codificar):

In [26]:
y_test = dict_dataset['test']['label']
y_test[:5]

['2. derogation',
 '2. derogation',
 '2. derogation',
 '2. derogation',
 '2. derogation']

In [27]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true = y_test, y_pred = y_pred, target_names=LABELS))
print(confusion_matrix(y_test, y_pred))


                                          precision    recall  f1-score   support

1. threats, plans to harm and incitement       0.61      0.47      0.53        89
                           2. derogation       0.56      0.74      0.64       454
                            3. animosity       0.56      0.44      0.49       333
               4. prejudiced discussions       0.61      0.30      0.40        94

                                accuracy                           0.57       970
                               macro avg       0.58      0.49      0.51       970
                            weighted avg       0.57      0.57      0.55       970

[[ 42  29  14   4]
 [ 15 335  92  12]
 [  7 179 145   2]
 [  5  53   8  28]]


Como se ha dicho antes, los resultados van a cambiar según la tarea y el modelo.

Si por ejemplo, estamos usando el modelo BERT con textos aumentados para la tarea b, la macro F1 es de 51%, que es 21 puntos superior a la macro F1 obtenido por el mismo modelo para la misma tarea pero sin datos aumentados.

Por tanto, podemos decir, que para la tarea b (y el modelo BERT), el uso de técnicas de DA sí ayudan a mejorar los resultados.


Puedes almacenar los resultados para poder compararlos con lso de los otros modelos:

In [28]:
import os
output_dir = 'results/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

clsf_report = pd.DataFrame(classification_report(y_true = y_test, y_pred = y_pred, target_names=LABELS, output_dict=True)).transpose()
path_results = output_dir+model_name

clsf_report.to_csv(path_results+'_{}.csv'.format(TASK), index= True)
print(path_results+'_{}.csv'.format(TASK), ' grabado!')


results/bert-base-uncased_b.csv  grabado!


Ajusta cada uno de los modelos para cada una de las tareas, y almacena sus resultados.
¿Qué modelo es el mejor para cada tarea?, ¿cuál es la mejor f1 para cada tarea?. También es interesante que discutas los resultados para las clases en las tareas B y C.